## users 테이블에 패스워드 넣기

In [1]:
import json
import pymysql

In [2]:
with open('mysql.json','r') as file:
    config_str = file.read()
config = json.loads(config_str)

In [3]:
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port'],
)

- 패스워드 암호화

In [4]:
import hashlib
import base64

- Secure Hash Algorithm(SHA-256)

In [5]:
pwd = '1234'
pwd_sha256 = hashlib.sha256(pwd.encode())
pwd_sha256.digest()

b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'

- Base64로 인코딩

In [6]:
base64.b64encode(pwd_sha256.digest())

b'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [10]:
hashed_pwd= base64.b64encode(pwd_sha256.digest()).decode('utf-8')
hashed_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [11]:
len(hashed_pwd)

44

In [12]:
# 패스워드를 생성하는 함수
def gen_pwd(pwd):
    pwd_sha256 = hashlib.sha256(pwd.encode())
    hashed_pwd= base64.b64encode(pwd_sha256.digest()).decode('utf-8')
    return hashed_pwd

### users table을 읽어서 pwd 열에 패스워드를 채움

In [13]:
sql = 'SELECT uid FROM users;'
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()

conn.commit()

In [15]:
sql= 'UPDATE users SET pwd=%s where uid=%s;'
cur=conn.cursor()
for row in results:
    uid = row[0]
    pwd = gen_pwd(uid+'12')
    cur.execute(sql,(pwd,uid))
conn.commit()

### 로그인 

- 1. uid OK, pwd OK

In [18]:
uid,pwd = 'admin','admin12'
sql = 'SELECT uid, pwd FROM users WHERE uid=%s;'
cur = conn.cursor()
cur.execute(sql,(uid,))
result = cur.fetchone()
db_pwd = result[1]
db_pwd

'EUZjqxlO3LP2HUCYg85K5sPC+YVBlAlaU4UBHRW+y+8='

In [19]:
hashed_pwd = gen_pwd(pwd)
hashed_pwd

'EUZjqxlO3LP2HUCYg85K5sPC+YVBlAlaU4UBHRW+y+8='

- 2) uid OK, pwd BAD

In [20]:
pwd = '1234'
hashed_pwd = gen_pwd(pwd)
hashed_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

- 3) uid BAD

In [21]:
uid = 'fool'

In [22]:
sql = 'SELECT uid,pwd FROM users WHERE uid=%s;'
cur = conn.cursor()
cur.execute(sql,(uid,))

0

In [24]:
result = cur.fetchone()

In [25]:
result

- 1), 2), 3) 을 하나로 합치기

In [26]:
uid, pwd = 'admin', 'admin1234'

In [28]:
sql = 'SELECT uid, pwd FROM users WHERE uid=%s and is_deleted=0;'
cur = conn.cursor()
cur.execute(sql,(uid,))
result = cur.fetchone()

if result:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('correct')
    else: 
        print('incorrect password') 

else:
    print('BAD uid')

incorrect password


In [30]:
## if문 틈새 특강 ##

x = ()  # '' [] 등 비어있는 것 부여하면
if x:   # x가 존재하면 이라는 뜻임
    print('true')
else: 
    print('false')

false


In [ ]:
## 아래와 같이 다른 방법으로 할 수도 있음

In [32]:
uid, pwd = 'admin12', 'admin1234'

In [33]:
cur = conn.cursor()
cur.execute(sql,(uid,))
result = cur.fetchone()
try:
    db_pwd = result[1]    #빈것에 인덱스를 걸었으므로 오류 나서 except로 넘어감
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('correct')
    else: 
        print('incorrect password') 

except:
    print('BAD uid')

BAD uid


In [ ]:
# 모두 끝나고 나면 이렇게 실행하기
cur.close()
conn.close()